# Linear Solver Performance: Constant Coefficient Poisson, Single Core

### Part 2, Evaluation

In [1]:
#r "C:\Users\jenkinsci\Documents\BoSSS-kummer\public\src\L4-application\BoSSSpad\bin\Debug\net5.0\BoSSSpad.dll"
//#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Application.SipPoisson;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
wmg.Init("LinslvPerfSer");
wmg.SetNameBasedSessionJobControlCorrelation();

Project name is set to 'LinslvPerfSer'.
Opening existing database '\\fdygitrunner\ValidationTests\LinslvPerfSer'.


In [3]:
// Manually add database (e.g. when accessing the database of user `JenkinsCI` as some other, regular guy)
//var tempDb = OpenOrCreateDatabase(@"\\fdygitrunner\ValidationTests\LinslvPerf_ConstPoissonMpi1");

In [4]:
databases

#0: { Session Count = 241; Grid Count = 30; Path = \\fdygitrunner\ValidationTests\LinslvPerfSer }


In [5]:
wmg.Sessions

#0: LinslvPerfSer	BottiPietroStokes2D-J147456_p2_exp_Kcycle_schwarz	04/15/2022 10:31:35	6ef09c97...
#1: LinslvPerfSer	XdgStokes-J4096_p5_direct_pardiso	04/15/2022 09:19:07	7042c09e...
#2: LinslvPerfSer	BottiPietroStokes2D-J65536_p3_pMultigrid	04/15/2022 11:41:00	49c171e1...
#3: LinslvPerfSer	XdgStokes-J13824_p3_direct_pardiso	04/15/2022 09:17:35	5ab74bfc...
#4: LinslvPerfSer	BottiPietroStokes2D-J36864_p3_exp_Kcycle_schwarz	04/15/2022 10:45:44	b4b66d1a...
#5: LinslvPerfSer	BottiPietroStokes2D-J16384_p3_pMultigrid	04/15/2022 11:36:47	f8e2ebdd...
#6: LinslvPerfSer	BottiPietroStokes2D-J36864_p5_exp_Kcycle_schwarz	04/15/2022 11:02:01	37356940...
#7: LinslvPerfSer	BottiPietroStokes2D-J65536_p3_exp_Kcycle_schwarz	04/15/2022 10:47:37	d8c4c789...
#8: LinslvPerfSer	BottiPietroStokes2D-J65536_p2_exp_Kcycle_schwarz	04/15/2022 10:29:54	e44f7e0e...
#9: LinslvPerfSer	XdgStokes-J13824_p3_exp_Kcycle_schwarz	04/15/2022 09:37:28	937cb36a...
#10: LinslvPerfSer	XdgPoisson-J262144_k2_pMultigrid	04/15/2022 0

In [6]:
var FailedSessions = wmg.Sessions.Where(Si => Si.SuccessfulTermination == false
                                              || Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0);
FailedSessions

#0: LinslvPerfSer	BottiPietroStokes2D-J36864_p5_pMultigrid*	04/15/2022 11:58:00	649b5990...
#1: LinslvPerfSer	BottiPietroStokes2D-J36864_p3_pMultigrid*	04/15/2022 11:38:51	efdfbc10...
#2: LinslvPerfSer	BottiPietroStokes2D-J147456_p2_pMultigrid*	04/15/2022 11:22:19	8eb392f7...
#3: LinslvPerfSer	BottiPietroStokes2D-J65536_p2_pMultigrid*	04/15/2022 11:20:07	1db29272...
#4: LinslvPerfSer	BottiPietroStokes2D-J36864_p2_pMultigrid*	04/15/2022 11:18:04	32b6d3af...
#5: LinslvPerfSer	BottiPietroStokes2D-J16384_p2_pMultigrid*	04/15/2022 11:16:00	eb02c22c...
#6: LinslvPerfSer	BottiPietroStokes2D-J2304_p2_pMultigrid*	04/15/2022 11:11:52	f71356c5...
#7: LinslvPerfSer	XdgStokes-J13824_p5_exp_Kcycle_schwarz*	04/15/2022 09:42:51	c15bdba3...
#8: LinslvPerfSer	XdgStokes-J32768_p3_exp_Kcycle_schwarz*	04/15/2022 09:38:48	c25a2af3...
#9: LinslvPerfSer	XdgPoisson-J32768_k5_pMultigrid*	04/15/2022 09:33:11	586d9894...
#10: LinslvPerfSer	XdgStokes-J13824_p5_exp_gmres_levelpmg*	04/15/2022 09:29:08	48c74f4d...
#1

In [7]:
//NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not terminate successfully.");

Unhandled exception: NUnit.Framework.AssertionException:   Some Sessions did not terminate successfully.
  Expected: 0
  But was:  12

   at NUnit.Framework.Assert.ReportFailure(String message) in /_/src/NUnitFramework/framework/Assert.cs:line 395
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.cs:line 383
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.That.cs:line 229
   at NUnit.Framework.Assert.Zero(Int32 actual, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.Conditions.cs:line 484
   at Submission#9.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exce

In [ ]:
//FailedSessions.ForEach(s => s.Delete(true));

## Create Table for Post-Processing

The timeing information that is requires for this study is not pesent in the default session table.
Instead, it must be extracted from the *session profiling*.

To obtain timing-measurements, the instrumentation generated by certain `BlockTrace`-blocks within the `UniSolver` class is extracted and added to the data table:

In [8]:
// evaluators to add additional columns to the session table
static class AddCols {
    static int GetSolType(ISessionInfo SI) {
        if(SI.GetControl().GetSolverType().Name.Contains("XNSE"))
            return 3;
        if(SI.GetControl().GetSolverType().Name.Contains("XdgPoisson3Main"))
            return 2;
        return 1;
    }
    
    
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        ilPSP.Tracing.MethodCallRecord nd;
        if(GetSolType(SI) == 3)
            nd = mcr.FindChildren("*.XdgTimestepping.ComputeOperatorMatrix").Single();
        else 
            nd  = mcr.FindChildren("MatrixAssembly").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Aggregation_basis_init").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Solver_Init").Single();
        //var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var nd  = mcr.FindChildren("Solver_Run").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_InitAndRun_time(ISessionInfo SI) {
        double agitime = (double) Aggregation_basis_init_time(SI);
        double initime = (double) Solver_Init_time(SI);
        double runtime = (double) Solver_Run_time(SI);
        
        return agitime + initime + runtime;
    }
    static public object Solver_TimePerIter(ISessionInfo SI) {
        double runtime = (double) Solver_Run_time(SI);
        double NoOfItr = Convert.ToDouble(SI.KeysAndQueries["NoIter"]);
        return runtime/NoOfItr;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
    }
    static public object ComputeNodeName(ISessionInfo SI) {
        return SI.ComputeNodeNames.First();
    }
    
    //static public object DOFs(ISessionInfo SI) {
    //    int NoOfItr = Convert.ToInt32(SI.KeysAndQueries["NoIter"]);
    //    return NoOfItr;
    //}
    
    static public object DGdegree(ISessionInfo SI) {
        switch(GetSolType(SI)) {
            case 3:
            return SI.KeysAndQueries.Single(kv => kv.Key.Contains("DGdegree:Velocity")).Value;
            case 2:
            return SI.KeysAndQueries.Single(kv => kv.Key == "DGdegree:u").Value;    
            case 1:
            return SI.KeysAndQueries.Single(kv => kv.Key == "DGdegree:T").Value;
        }
        throw new ArgumentException();
    }
}

In [9]:
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("Solver_InitAndRun_time", AddCols.Solver_InitAndRun_time);
wmg.AdditionalSessionTableColums.Add("Solver_TimePerIter", AddCols.Solver_TimePerIter);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);
wmg.AdditionalSessionTableColums.Add("ComputeNodeName", AddCols.ComputeNodeName);
wmg.AdditionalSessionTableColums.Add("DGdegree", AddCols.DGdegree);
//wmg.AdditionalSessionTableColums.Add("DOFs", AddCols.DOFs);

In [10]:
var SessTab = wmg.SessionTable;

//// The Session column can't be serialized,
//// we have to remove it
//List<string> AllCols = FullSessTab.GetColumnNames().ToList();
//AllCols.Remove("Session");
//FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());

In [11]:
//wmg.Sessions[0].KeysAndQueries["NoIter"].GetType()

In [12]:
//wmg.Sessions[0].Name

In [13]:
//SessTab.GetColumn("LinearSolver.Name")

In [14]:
SessTab.GetColumnNames().ToConcatString("", "; ", "")

ProjectName; SessionName; DGdegree:Velocity*; DGdegree:Pressure; DGdegree:PhiDG; DGdegree:Phi; DGdegree:Phi2DG; DGdegree:Phi2; DGdegree:Curvature; Bndtype:wall; UseImmersedBoundary; TimesteppingMode; Timestepper_LevelSetHandling; GeneratedFromCode; ControlFileText_Index; NoOfMultigridLevels; InitialValues_EvaluatorsVec[0].Key; InitialValues_EvaluatorsVec[1].Key; InitialValues_Evaluators_TimeDep[0].Key; InitialValues_Evaluators_TimeDep[1].Key; dtFixed; UseSchurBlockPrec; RefineStrategy; BaseRefinementLevel; RefinementLevel; RefineNavierSlipBoundary; ClearVelocitiesOnRestart; ReInitOnRestart; adaptiveReInit; InitSignedDistance; AdvancedDiscretizationOptions.ViscosityMode; AdvancedDiscretizationOptions.LFFA; AdvancedDiscretizationOptions.LFFB; AdvancedDiscretizationOptions.PenaltySafety; AdvancedDiscretizationOptions.SurfStressTensor; AdvancedDiscretizationOptions.STFstabilization; AdvancedDiscretizationOptions.SetSurfaceTensionMaxValue; AdvancedDiscretizationOptions.freeSurfaceFlow; Adva

Select those columns which are of interest:

In [16]:
var SubTab = SessTab.ExtractColumns(
    "SessionName","DGdegree", "Grid:NoOfCells", "LinearSolver.Name", "LinearSolver.Shortname", "DOFs", "MatrixAssembly",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "Solver_InitAndRun_time", "NoIter", 
    "Solver_TimePerIter", "ComputeNodeName", "RegularTerminated");

In [17]:
// Filename
var now         = DateTime.Now;
string docName  = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
SubTab.SaveToFile(docName + ".json");
SubTab.ToCSVFile(docName + ".csv");

Only consider runs which have been successful:

In [18]:
SubTab = SubTab.ExtractRows(delegate(int iRow, IDictionary<string, object> row) {
    return (bool)(row["RegularTerminated"]);
});

In [19]:
//var SubTab = TableExtensions.LoadFromFile("temp.json");

In [20]:
//SubTab.Print();

## Vizualisation of Results

The following data is available:

In [21]:
SubTab.GetColumnNames()

index,value
0,SessionName
1,DGdegree
2,Grid:NoOfCells
3,LinearSolver.Name
4,LinearSolver.Shortname
5,DOFs
6,MatrixAssembly
7,Aggregation_basis_init_time
8,Solver_Init_time
9,Solver_Run_time


Available DG degrees:

In [22]:
var DGdegrees = SubTab.GetColumn<int>("DGdegree").ToSet().OrderBy(s => s).ToArray();
DGdegrees

index,value
0,2
1,3
2,5


All used solvers:

In [23]:
SubTab.GetColumn<string>("LinearSolver.Shortname").ToSet()

index,value
0,OrthoMG w Add Swz
1,PARDISO
2,Ortho w pmG
3,GMRES w p2G


In [24]:
const string Pardiso = "PARDISO";
const string OrmgSwz = "OrthoMG w Add Swz";
const string GmsrP2g = "GMRES w p2G";
const string OrmgPmg = "Ortho w pmG";
const string OrmgILU = "OrthoMG w ILU";

Used Cases:

In [25]:
const string Poisson = "SIP_Poisson";
const string XPoisson = "XdgPoisson";
const string Stokes2D = "BottiPietroStokes2D";
const string Stokes3D = "BottiPietroStokes3D";
const string XStokes = "XdgStokes";
string[] AllCases = new string[] { Poisson, XPoisson, Stokes2D, Stokes3D, XStokes };

In [26]:
string ExtractCase(string sessionName) {
    return AllCases.Single(caseName => sessionName.Contains(caseName));
}

### Macro Routine for Multiplot

The following routine combines the plotting code which is common for all sub-plot in this evaluation;
only the y-axis needs to be specified. 

In [27]:
PlotFormat SlvCode2Pltfmt(string solver_name, string caseName) { 
    var Fmt = new PlotFormat();
    switch(solver_name) {
        case Pardiso: 
            Fmt.PointType = PointTypes.OpenCircle;
            Fmt.DashType  = DashTypes.Dotted;
            break;
        case GmsrP2g: 
            Fmt.PointType = PointTypes.Box;
            break;
        case OrmgSwz: 
            Fmt.PointType = PointTypes.LowerTriangle;
            break;
        case OrmgILU:
            Fmt.PointType = PointTypes.Diamond;
            break;
        case OrmgPmg:
            Fmt.PointType = PointTypes.Asterisk;
            break;
        default:
            Console.WriteLine("unknown: " + solver_name); 
            break;
    } 
    //Console.WriteLine("name is: " + solver_name); 
    Fmt.PointSize = 0.85;
    Fmt.LineWidth = 2;    
    Fmt.Style     = Styles.LinesPoints;
    
    if(caseName.Contains(Poisson))
        Fmt.LineColor = LineColors.Green;
    else if(caseName.Contains(XPoisson))
        Fmt.LineColor = LineColors.Blue;
    else if(caseName.Contains(Stokes2D))
        Fmt.LineColor = LineColors.Orange;
    else if(caseName.Contains(XStokes))
        Fmt.LineColor = LineColors.Red;
    

    return Fmt;
}

The following function will later be used to check 

In [28]:
void AssertSlope(Plot2Ddata plot, string caseName, string solverName, double allowedSlope) {
   NUnit.Framework.Assert.LessOrEqual(
      plot.Regression().Single(tt =>  tt.Key.Contains(caseName) && tt.Key.Contains(solverName)).Value,
      1.1,
      $"{solverName} scaling for {caseName} exceeds limit");
}

In [29]:
Plot2Ddata[,] PlotSolverBehave(string Yname, bool LogY, double yMin, double yMax, double LegendYpos) {

int rows    = DGdegrees.Length;
int columns = 1;
string[] ignore_solvers = {};
Plot2Ddata[,] multiplots = new Plot2Ddata[rows + 1,columns];
int pDegree = 0;
for(int iRow = 0; iRow < rows; iRow++) {
for(int iCol = 0; iCol < columns; iCol++) {
    
    if(pDegree > rows*columns-1)
        continue;
    int tmpDG = -1;
    if(pDegree < DGdegrees.Length)
        tmpDG = DGdegrees[pDegree];
    
    //Create Graphs
    multiplots[iRow,iCol] = SubTab.ToPlot("DOFs", Yname, // column for x- and y
       delegate (int iTabRow, 
                 IDictionary<string, object> Row, 
                 out string Nmn, 
                 out PlotFormat Fmt) { 
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           // PlotRowSelector:
           // selects, which table row goes to which graph,
           // and the respective color  
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           int k = Convert.ToInt32(Row["DGdegree"]);
           if(k != tmpDG) {
                // degree does not match -> not in this plot
                Nmn = null;
                Fmt = null;
                return;
           }
 
           string solver_name = (string) (Row["LinearSolver.Shortname"]);
           //ignore the solvers specified in ingore_solvers
           foreach(string sc in ignore_solvers){
               if(solver_name == sc){
                   System.Console.WriteLine("skipped");
                   Nmn = null;
                   Fmt = null;
               return;
               }
           }
           
           
           //process the other solvers
           string caseName = ExtractCase( (string) Row["SessionName"]);
           Fmt = SlvCode2Pltfmt(solver_name, caseName);
           Nmn = solver_name + "/" + caseName;
       });
    
       double[] dof = new[]{1e3,1e6};
       double[] linT = dof.Select(x => x*0.001).ToArray();
       var linP = new Plot2Ddata.XYvalues("linear", dof, linT);
       linP.Format.FromString("- black");
       ArrayTools.AddToArray(linP, ref multiplots[iRow,iCol].dataGroups);
 
       //all about axis
       string Title = string.Format("$k = {0}$", tmpDG);
       multiplots[iRow,iCol].Ylabel = Title;
       multiplots[iRow,iCol].LogX = true;
       multiplots[iRow,iCol].LogY = LogY;
    
       //specify range of axis
       multiplots[iRow,iCol].YrangeMin = yMin;
       multiplots[iRow,iCol].YrangeMax = yMax;
       multiplots[iRow,iCol].XrangeMin = 1e2;
       multiplots[iRow,iCol].XrangeMax = 1e7;
    
       //multiplots[iRow,iCol].Y2rangeMin = 1e-3;
       //multiplots[iRow,iCol].Y2rangeMax = 1e+4;
       //multiplots[iRow,iCol].X2rangeMin = 1e2;
       //multiplots[iRow,iCol].X2rangeMax = 1e7;
    
       //spacing around plots
       multiplots[iRow,iCol].ShowLegend = false;
       multiplots[iRow,iCol].tmargin = 0;
       multiplots[iRow,iCol].bmargin = 2;
       multiplots[iRow,iCol].lmargin = 5;
       multiplots[iRow,iCol].rmargin = 5;
       multiplots[iRow,iCol].ShowXtics = false;

       //I am legend ...
       if(iRow == rows - 1) {
          multiplots[iRow,iCol].ShowLegend = true;
          //multiplots[iRow,iCol].LegendAlignment = new string[]{"o", "r", "t" };
          multiplots[iRow,iCol].LegendFont = 12;
          multiplots[iRow,iCol].Legend_maxrows = 100;
           multiplots[iRow,iCol].LegendPosition = new double[] { 4e7, LegendYpos };
          //multiplots[iRow,iCol].LegendSwap  = true;
       }
       //and i am special ...
       if(iRow == rows - 1)
           multiplots[iRow,iCol].ShowXtics = true;
    pDegree++;
}                        
}
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
return multiplots;
}

### Total Runtime

One would expect **linear scaling** with grid resolution; it may scale nonlinear with DG polynomial order.

In [30]:
var multiplots = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4, 1e-5);
//multiplots.PlotCairolatex().PlotNow()
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.ToGnuplot().PlotSVG(xRes:800,yRes:1200)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe



warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 7 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 OrthoMG w Add Swz/SIP P oisson 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 Ortho w pmG/SIP P oisson 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 GMRES w p2G/SIP P oisson 
 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 PARDISO/SIP P oisson 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 linear

In [31]:
var multiplots = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4, 1e-5);
multiplots.PlotCairolatex().SaveTo("Runtime.tex");

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


In [32]:
/*var legend = new Plot2Ddata();
legend.AddDataGroup("Pardiso",new double[] {1,2},new double[] {1,2}, "-og");
legend.ShowXtics = false;
legend.ShowXtics = false;
//legend.XrangeMin = 3;
//legend.XrangeMax = 3.1;

//legend.AddGnuplotCommand("set border 0");
//legend.AddGnuplotCommand("set style line 101 lc rgb '#808080' lt 1 lw 1");
legend.AddGnuplotCommand("unset xlabel");
legend.AddGnuplotCommand("unset ylabel");
legend.AddGnuplotCommand("set format x ''");
legend.AddGnuplotCommand("set format y ''");
legend.AddGnuplotCommand("set tics scale 0");

legend.PlotNow()*/

#### Regression and Error checking (Runtime)

For polynomial degree 2:

In [33]:
multiplots[0,0].Regression()

index,Key,Value
0,OrthoMG w Add Swz/BottiPietroStokes2D,1.5970335856716689
1,Ortho w pmG/XdgPoisson,1.0437229807123838
2,OrthoMG w Add Swz/XdgStokes,1.7530238914092107
3,Ortho w pmG/BottiPietroStokes2D,1.1685765652167825
4,PARDISO/XdgStokes,2.1775399826092996
5,GMRES w p2G/XdgStokes,1.3642948720863632
6,GMRES w p2G/XdgPoisson,0.8101905875930039
7,GMRES w p2G/BottiPietroStokes2D,0.7646129066287745
8,OrthoMG w Add Swz/XdgPoisson,0.7594369287270664
9,PARDISO/BottiPietroStokes2D,0.7972193452382337


For polynomial degree 3:

In [34]:
multiplots[1,0].Regression()

index,Key,Value
0,Ortho w pmG/BottiPietroStokes2D,1.3122075674480311
1,PARDISO/XdgStokes,2.167117508863168
2,OrthoMG w Add Swz/BottiPietroStokes2D,1.5812640046137696
3,OrthoMG w Add Swz/XdgStokes,1.6256838718029405
4,GMRES w p2G/BottiPietroStokes2D,0.9128427873002538
5,PARDISO/XdgPoisson,1.162785507733558
6,Ortho w pmG/XdgPoisson,1.1011789631069506
7,GMRES w p2G/XdgStokes,1.0785279133479444
8,OrthoMG w Add Swz/XdgPoisson,0.909389825967447
9,PARDISO/BottiPietroStokes2D,0.9801748654136567


For polynomial degree 5:

In [35]:
multiplots[2,0].Regression()

index,Key,Value
0,PARDISO/XdgStokes,1.9540600167988844
1,OrthoMG w Add Swz/BottiPietroStokes2D,1.2809257022927736
2,Ortho w pmG/BottiPietroStokes2D,1.1032666031788723
3,OrthoMG w Add Swz/XdgStokes,0.9127883094756752
4,GMRES w p2G/XdgStokes,1.1307558791713368
5,GMRES w p2G/BottiPietroStokes2D,1.0754432667871854
6,GMRES w p2G/XdgPoisson,1.0140417581724932
7,OrthoMG w Add Swz/XdgPoisson,0.9989088303220051
8,Ortho w pmG/XdgPoisson,1.3945119975476505
9,PARDISO/XdgPoisson,1.3932767952991794


In [36]:
AssertSlope(multiplots[0,0], Poisson, GmsrP2g, 1.1);
AssertSlope(multiplots[1,0], Poisson, GmsrP2g, 1.1);
AssertSlope(multiplots[2,0], Poisson, GmsrP2g, 1.1);

Unhandled exception: NUnit.Framework.AssertionException:   GMRES w p2G scaling for SIP_Poisson exceeds limit
  Expected: less than or equal to 1.1000000000000001d
  But was:  1.1323198532974839d

   at NUnit.Framework.Assert.ReportFailure(String message) in /_/src/NUnitFramework/framework/Assert.cs:line 395
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.cs:line 383
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.That.cs:line 229
   at NUnit.Framework.Assert.LessOrEqual(Double arg1, Double arg2, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.Comparisons.cs:line 943
   at Submission#30.AssertSlope(Plot2Ddata plot, String caseName, String solverName, Double allowedSlope)
   at Submission#38.<<Initialize>>d__0.MoveNext()
--- End of stack trace from 

Experimental: Latex Preview:

In [37]:
//var multiplots = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-2, 1e+4);
//multiplots.PlotCairolatex().PlotNow()

### Number of Iterations

For the iterative solvers, obe would expect **almost constant number of solver iterations**, 
independent of grid resolution. It may, however depend in nonlinar fashion on the DG polynomial order.

In [38]:
var multiplots = PlotSolverBehave("NoIter", false, 0, 300, -150);
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.ToGnuplot().PlotSVG(xRes:800,yRes:1200)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe



warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 300 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 300 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 7 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 OrthoMG w Add Swz/SIP P oisson 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 Ortho w pmG/SIP P oisson 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 GMRES w p2G/SIP P oisson 
 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 PARDISO/SIP P oisson 
 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 linear

In [39]:
multiplots[0,0].dataGroups

index,Format,Name,UseX2,UseY2,Abscissas,Values
0,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/BottiPietroStokes2D,False,False,"[ 960, 3840, 8640, 15360, 34560, 61440, 245760, 552960, 983040, 2211840 ]","[ 1, 1, 1, 1, 39, 103, 247, 452, 595, 866 ]"
1,BoSSS.Solution.Gnuplot.PlotFormat,Ortho w pmG/XdgPoisson,False,False,"[ 160, 880, 5920, 43840, 339040, 2671600 ]","[ 1, 3, 6, 11, 36, 55 ]"
2,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/XdgStokes,False,False,"[ 18224, 142528, 478992, 1128800 ]","[ 1, 136, 207, 287 ]"
3,BoSSS.Solution.Gnuplot.PlotFormat,Ortho w pmG/BottiPietroStokes2D,False,False,"[ 960, 3840, 8640, 15360, 61440 ]","[ 8, 21, 32, 46, 93 ]"
4,BoSSS.Solution.Gnuplot.PlotFormat,PARDISO/XdgStokes,False,False,"[ 18224, 142528, 478992 ]","[ 1, 1, 1 ]"
5,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G/XdgStokes,False,False,"[ 18224, 142528, 478992, 1128800 ]","[ 144, 178, 202, 212 ]"
6,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G/XdgPoisson,False,False,"[ 160, 880, 5920, 43840, 339040, 2671600 ]","[ 3, 29, 42, 62, 62, 55 ]"
7,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G/BottiPietroStokes2D,False,False,"[ 960, 3840, 8640, 15360, 34560, 61440, 245760, 552960, 983040, 2211840 ]","[ 59, 70, 71, 72, 72, 71, 69, 67, 66, 64 ]"
8,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/XdgPoisson,False,False,"[ 160, 880, 5920, 43840, 339040, 2671600 ]","[ 1, 1, 1, 8, 16, 18 ]"
9,BoSSS.Solution.Gnuplot.PlotFormat,PARDISO/BottiPietroStokes2D,False,False,"[ 960, 3840, 8640, 15360, 34560, 61440, 245760, 552960, 983040 ]","[ 1, 1, 1, 1, 1, 1, 1, 1, 1 ]"


In [40]:
multiplots[1,0].dataGroups

index,Format,Name,UseX2,UseY2,Abscissas,Values
0,BoSSS.Solution.Gnuplot.PlotFormat,Ortho w pmG/BottiPietroStokes2D,False,False,"[ 1664, 6656, 14976, 26624, 59904, 106496, 425984, 1703936 ]","[ 9, 9, 7, 7, 11, 11, 13, 16 ]"
1,BoSSS.Solution.Gnuplot.PlotFormat,PARDISO/XdgStokes,False,False,"[ 37520, 293440, 986160 ]","[ 1, 1, 1 ]"
2,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/BottiPietroStokes2D,False,False,"[ 1664, 6656, 14976, 26624, 59904, 106496, 425984, 958464, 1703936 ]","[ 1, 1, 1, 11, 63, 106, 331, 513, 241 ]"
3,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/XdgStokes,False,False,"[ 37520, 293440, 986160 ]","[ 20, 97, 189 ]"
4,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G/BottiPietroStokes2D,False,False,"[ 1664, 6656, 14976, 26624, 59904, 106496, 425984, 958464, 1703936 ]","[ 137, 147, 155, 175, 195, 203, 213, 219, 223 ]"
5,BoSSS.Solution.Gnuplot.PlotFormat,PARDISO/XdgPoisson,False,False,"[ 320, 1760, 11840, 87680, 678080 ]","[ 1, 1, 1, 1, 1 ]"
6,BoSSS.Solution.Gnuplot.PlotFormat,Ortho w pmG/XdgPoisson,False,False,"[ 320, 1760, 11840, 87680, 678080 ]","[ 3, 4, 6, 11, 14 ]"
7,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G/XdgStokes,False,False,"[ 37520, 293440, 986160 ]","[ 283, 335, 337 ]"
8,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/XdgPoisson,False,False,"[ 320, 1760, 11840, 87680, 678080 ]","[ 1, 1, 1, 15, 20 ]"
9,BoSSS.Solution.Gnuplot.PlotFormat,PARDISO/BottiPietroStokes2D,False,False,"[ 1664, 6656, 14976, 26624, 59904, 106496, 425984, 958464 ]","[ 1, 1, 1, 1, 1, 1, 1, 1 ]"


In [41]:
multiplots[2,0].dataGroups

index,Format,Name,UseX2,UseY2,Abscissas,Values
0,BoSSS.Solution.Gnuplot.PlotFormat,PARDISO/XdgStokes,False,False,"[ 108808, 850976 ]","[ 1, 1 ]"
1,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/BottiPietroStokes2D,False,False,"[ 3648, 14592, 32832, 58368, 131328, 233472, 933888, 2101248 ]","[ 1, 1, 17, 10, 63, 16, 22, 223 ]"
2,BoSSS.Solution.Gnuplot.PlotFormat,Ortho w pmG/BottiPietroStokes2D,False,False,"[ 3648, 14592, 32832, 58368, 131328, 233472 ]","[ 6, 6, 6, 7, 7, 8 ]"
3,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/XdgStokes,False,False,"[ 108808, 850976 ]","[ 35, 62 ]"
4,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G/XdgStokes,False,False,"[ 108808, 850976 ]","[ 587, 878 ]"
5,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G/BottiPietroStokes2D,False,False,"[ 3648, 14592, 32832, 58368, 131328, 233472, 933888, 2101248 ]","[ 238, 275, 301, 343, 422, 459, 524, 518 ]"
6,BoSSS.Solution.Gnuplot.PlotFormat,GMRES w p2G/XdgPoisson,False,False,"[ 896, 4928, 33152, 245504, 1898624 ]","[ 3, 163, 145, 159, 157 ]"
7,BoSSS.Solution.Gnuplot.PlotFormat,OrthoMG w Add Swz/XdgPoisson,False,False,"[ 896, 4928, 33152, 245504, 1898624 ]","[ 1, 1, 6, 11, 11 ]"
8,BoSSS.Solution.Gnuplot.PlotFormat,Ortho w pmG/XdgPoisson,False,False,"[ 896, 4928, 33152, 245504 ]","[ 3, 5, 7, 21 ]"
9,BoSSS.Solution.Gnuplot.PlotFormat,PARDISO/XdgPoisson,False,False,"[ 896, 4928, 33152, 245504 ]","[ 1, 1, 1, 1 ]"


### Runtime per Iteration

One would expect **linear scaling** with grid resolution; it may scale nonlinear with DG polynomial order.

In [42]:
var multiplots = PlotSolverBehave("Solver_TimePerIter", true, 1e-2, 1e+6, 1e-3);
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.ToGnuplot().PlotSVG(xRes:800,yRes:1200)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe



warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 
 
 
 
 
 PARDISO/SIPPoisson 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 
 
 
 
 
 linear 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 -1 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 5 $ 
 
 
 
 
 $10 6 $ 
 
 
 
 
 $10 7 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 PARDISO/XdgStokes 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 OrthoMG w Add Swz/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 Ortho w pmG/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 OrthoMG w Add Swz/XdgStokes 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgStokes 
 
 
 GMRES w p2G/XdgStokes 
 
 
 
 
 
 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 GMRES w p2G/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 GMRES w p2G/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 OrthoMG w Add Swz/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 Ortho w pmG/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 PARDISO/XdgPoisson 
 
 
 PARDISO/XdgPoisson 
 
 
 
 
 
 
 
 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 PARDISO/BottiPietroStokes2D 
 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthoMG w Add Swz/SIPPoisson 
 
 
 OrthoMG w Add Swz/SIP P oisson 
 
 
 
 
 
 
 
 
 
 Ortho w pmG/SIPPoisson 
 
 
 Ortho w pmG/SIP P oisson 
 
 
 
 
 
 
 
 
 
 GMRES w p2G/SIPPoisson 
 
 
 GMRES w p2G/SIP P oisson

In [43]:
multiplots[0,0].Regression()

index,Key,Value
0,OrthoMG w Add Swz/BottiPietroStokes2D,0.5425077743148222
1,Ortho w pmG/XdgPoisson,0.6638237330322476
2,OrthoMG w Add Swz/XdgStokes,0.42290974705068224
3,Ortho w pmG/BottiPietroStokes2D,0.6570196597351787
4,PARDISO/XdgStokes,2.230722672209828
5,GMRES w p2G/XdgStokes,1.2935427612193786
6,GMRES w p2G/XdgPoisson,0.564321166432723
7,GMRES w p2G/BottiPietroStokes2D,0.7500577896267171
8,OrthoMG w Add Swz/XdgPoisson,0.3821640463984916
9,PARDISO/BottiPietroStokes2D,0.7955069447394034


In [44]:
multiplots[1,0].Regression()

index,Key,Value
0,Ortho w pmG/BottiPietroStokes2D,1.2192297852676552
1,PARDISO/XdgStokes,2.202901837352027
2,OrthoMG w Add Swz/BottiPietroStokes2D,0.5833998585687655
3,OrthoMG w Add Swz/XdgStokes,0.9688036718277345
4,GMRES w p2G/BottiPietroStokes2D,0.8531825650360334
5,PARDISO/XdgPoisson,1.1923432779649832
6,Ortho w pmG/XdgPoisson,0.9438393813793271
7,GMRES w p2G/XdgStokes,1.0314962255828224
8,OrthoMG w Add Swz/XdgPoisson,0.4487612481124722
9,PARDISO/BottiPietroStokes2D,0.9972909869433692


In [45]:
multiplots[2,0].Regression()

index,Key,Value
0,PARDISO/XdgStokes,1.9775252341770764
1,OrthoMG w Add Swz/BottiPietroStokes2D,0.5677306940449904
2,Ortho w pmG/BottiPietroStokes2D,1.0489653829203636
3,OrthoMG w Add Swz/XdgStokes,0.6917417299962516
4,GMRES w p2G/XdgStokes,0.9386221140034563
5,GMRES w p2G/BottiPietroStokes2D,0.9580209193785851
6,GMRES w p2G/XdgPoisson,0.6164087416417758
7,OrthoMG w Add Swz/XdgPoisson,0.6467179887906765
8,Ortho w pmG/XdgPoisson,1.1630731986066771
9,PARDISO/XdgPoisson,1.4554702436389233


In [46]:
AssertSlope(multiplots[0,0], Poisson, GmsrP2g, 1.1);
AssertSlope(multiplots[1,0], Poisson, GmsrP2g, 1.1);
AssertSlope(multiplots[2,0], Poisson, GmsrP2g, 1.1);

## Appendix: Table Form of Extracted Data

In [47]:
SubTab.Print()

     SessionName                                       DGdegree Grid:NoOfCells LinearSolver.Name                                          LinearSolver.Shortname DOFs    MatrixAssembly Aggregation_basis_init_time Solver_Init_time Solver_Run_time Solver_InitAndRun_time NoIter Solver_TimePerIter   ComputeNodeName                              RegularTerminated 
0:   BottiPietroStokes2D-J147456_p2_exp_Kcycle_schwarz 2        147456         OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      2211840 120.4211715    11.9831287                  209.6428743      55699.1771459   55920.8031489          866    64.31775651951502    hpccluster2.fdy.maschinenbau.tu-darmstadt.de True              
1:   XdgStokes-J4096_p5_direct_pardiso                 5        4096           Sparse direct solver PARDISO                               PARDISO                850976  1346.4315187   12.9669712                  113.8229241      36566.6021018   36693.3919971          1      36566

21:  BottiPietroStokes2D-J256_p3_pMultigrid            3        256            Orthonormalization with p-multi-grid preconditioner        Ortho w pmG            6656    0.6117233      0.0423496                   0.4061976        27.1128822      27.5614294             9      3.012542466666667    hpcluster3.fdy.maschinenbau.tu-darmstadt.de  True              
22:  BottiPietroStokes2D-J36864_p2_exp_Kcycle_schwarz  2        36864          OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      552960  12.0301016     1.5396689                   30.7863395       3272.6624682    3304.9884766           452    7.240403690707965    hpcluster3.fdy.maschinenbau.tu-darmstadt.de  True              
23:  BottiPietroStokes2D-J64_p3_pMultigrid             3        64             Orthonormalization with p-multi-grid preconditioner        Ortho w pmG            1664    0.4182022      0.0292623                   0.1858073        2.7568115       2.9718811              9      0.306

43:  BottiPietroStokes2D-J1024_p3_exp_Kcycle_schwarz   3        1024           OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      26624   1.981625       0.1230618                   1.797821         4.3450654       6.2659482              11     0.3950059454545455   hpccluster2.fdy.maschinenbau.tu-darmstadt.de True              
44:  BottiPietroStokes2D-J576_p3_exp_Kcycle_schwarz    3        576            OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      14976   1.0038102      0.109466                    0.9135984        2.367043        3.3901073999999998     1      2.367043             hpcluster3.fdy.maschinenbau.tu-darmstadt.de  True              
45:  XdgPoisson-J32768_k5_exp_gmres_levelpmg           5        32768          GMRES with p-two-grid preconditioner                       GMRES w p2G            1898624 1029.020524    412.8005407                 85.1366936       953.1046496     1451.0418839           157    6.070

65:  BottiPietroStokes2D-J256_p5_exp_gmres_levelpmg    5        256            GMRES with p-two-grid preconditioner                       GMRES w p2G            14592   1.5051812      0.1218225                   0.5969753        5.8132856       6.5320834              275    0.021139220363636364 hpcluster3.fdy.maschinenbau.tu-darmstadt.de  True              
66:  XdgStokes-J32768_p2_exp_gmres_levelpmg            2        32768          GMRES with p-two-grid preconditioner                       GMRES w p2G            1128800 487.6903002    3.0606546                   66.4553115       1538.4283176    1607.9442837           212    7.2567373471698104   hpccluster2.fdy.maschinenbau.tu-darmstadt.de True              
67:  XdgPoisson-J32768_k3_direct_pardiso               3        32768          Sparse direct solver PARDISO                               PARDISO                678080  76.6495226     13.5268126                  10.0323905       2556.9890653    2580.5482684000003     1      2556.

87:  XdgStokes-J4096_p3_direct_pardiso                 3        4096           Sparse direct solver PARDISO                               PARDISO                293440  95.1405089     1.2475663                   10.7204967       1474.6747642    1486.6428272           1      1474.6747642         hpcluster3.fdy.maschinenbau.tu-darmstadt.de  True              
88:  BottiPietroStokes2D-J16384_p2_exp_gmres_levelpmg  2        16384          GMRES with p-two-grid preconditioner                       GMRES w p2G            245760  6.0437337      0.7124114                   5.402329         20.1807173      26.2954577             69     0.29247416376811597  hpcluster3.fdy.maschinenbau.tu-darmstadt.de  True              
89:  BottiPietroStokes2D-J4096_p2_exp_gmres_levelpmg   2        4096           GMRES with p-two-grid preconditioner                       GMRES w p2G            61440   4.6233581      0.3267838                   3.0184274        9.7197304       13.0649416             71     0.136

109: XdgPoisson-J64_k5_pMultigrid                      5        64             Orthonormalization with p-multi-grid preconditioner        Ortho w pmG            4928    6.4516405      0.4580016                   0.988328         2.2428261       3.6891556999999997     5      0.44856522           hpcluster3.fdy.maschinenbau.tu-darmstadt.de  True              
110: XdgPoisson-J8_k5_pMultigrid                       5        8              Orthonormalization with p-multi-grid preconditioner        Ortho w pmG            896     1.8220156      0.1017815                   0.2251823        0.4225688       0.7495326              3      0.14085626666666667  hpcluster3.fdy.maschinenbau.tu-darmstadt.de  True              
111: XdgPoisson-J512_k3_pMultigrid                     3        512            Orthonormalization with p-multi-grid preconditioner        Ortho w pmG            11840   3.2957416      0.2484663                   0.6348569        3.002859        3.8861822              6      0.500

131: XdgPoisson-J4096_k3_exp_Kcycle_schwarz            3        4096           OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      87680   15.959439      1.6601693                   3.9493537        20.551818       26.161341              15     1.3701212            hpcluster3.fdy.maschinenbau.tu-darmstadt.de  True              
132: XdgPoisson-J8_k5_exp_Kcycle_schwarz               5        8              OrthonormalizationMultigrid with Additive Schwarz Smoother OrthoMG w Add Swz      896     2.9935354      0.0881167                   0.1354212        0.4261576       0.6496955              1      0.4261576            hpccluster2.fdy.maschinenbau.tu-darmstadt.de True              
133: XdgPoisson-J32768_k2_exp_gmres_levelpmg           2        32768          GMRES with p-two-grid preconditioner                       GMRES w p2G            339040  59.77739       6.360538                    11.6467732       85.4405973      103.4479085            62     1.378

153: BottiPietroStokes2D-J36864_p2_direct_pardiso      2        36864          Sparse direct solver PARDISO                               PARDISO                552960  10.9441623     1.0735519                   7.4506738        31.1738636      39.6980893             1      31.1738636           hpcluster3.fdy.maschinenbau.tu-darmstadt.de  True              
154: XdgStokes-J512_p2_exp_gmres_levelpmg              2        512            GMRES with p-two-grid preconditioner                       GMRES w p2G            18224   4.5698943      0.0464497                   0.5845306        3.9734251       4.6044054              144    0.02759322986111111  hpcluster3.fdy.maschinenbau.tu-darmstadt.de  True              
155: BottiPietroStokes2D-J64_p3_direct_pardiso         3        64             Sparse direct solver PARDISO                               PARDISO                1664    1.0256412      0.0387866                   0.1047105        0.5883402       0.7318373              1      0.588

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

